# 1.Import dependencies

In [5]:
import cv2
import joblib
import numpy as np
import pygame
from scipy.spatial import distance as dist
face_classifer = joblib.load("FaceClassifier_HoG(64, 64)(16, 16)(8, 8)(0.95).pkl")

# 3. Define Helper Functions

In [6]:
win_size = (64, 64) 
block_size = (16, 16)
block_stride = (8, 8)
cell_size = (8, 8)
num_bins = 9
yawn_counter = 0
HaarCascade=1

lower_color = np.array([100, 50, 0], dtype=np.uint8)
upper_color = np.array([205, 130, 130], dtype=np.uint8)


HOG = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)
# Initialize pygame mixer
pygame.mixer.init()
# Load the buzzer sound
buzzer_sound = pygame.mixer.Sound("cristiano-ronaldo-siuu-made-with-Voicemod-technology.mp3")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def calculate_mouth_aspect_ratio(mouth):
    x, y, w, h = cv2.boundingRect(mouth)
    aspect_ratio = w // h
    return aspect_ratio

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear


def extract_features_face_detection(Image):

    gray_image = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)

    equalized_image = cv2.equalizeHist(gray_image)

    resized_image = cv2.resize(equalized_image, win_size)
        
    feature = HOG.compute(resized_image).flatten()

    if feature.max() > 0:
        feature /= feature.max()

    return feature



def union_boxes_min_max(boxes, min_x, min_y, max_x, max_y):
  overall_box = None

  if not overall_box:
    for box in boxes:
      x_min, y_min, x_max, y_max = box
      min_x = min(min_x, x_min)
      min_y = min(min_y, y_min)
      max_x = max(max_x, x_max)
      max_y = max(max_y, y_max)
    overall_box = (min_x, min_y, max_x, max_y)
    
    return overall_box



def filter_boxes_with_skin_mask(boxes, skin_mask):
  filtered_boxes = []
  for box in boxes:
    x_min, y_min, x_max, y_max = box
    
    box_area = (x_max - x_min) * (y_max - y_min)
    
    intersection_mask = skin_mask[y_min:y_max, x_min:x_max]
    intersection_area = cv2.countNonZero(intersection_mask)
    
    iou = intersection_area / box_area

    # Define threshold for acceptable overlap with skin mask
    threshold = 0.5

    if iou >= threshold:
      filtered_boxes.append(box)

  return filtered_boxes  



def get_bounding_box(mask):
    non_zero_pixels = np.nonzero(mask)

    min_row, min_col = np.min(non_zero_pixels, axis=1)
    max_row, max_col = np.max(non_zero_pixels, axis=1)

    # Return coordinates in desired format (x0, y0, x1, y1)
    return min_col, min_row, max_col, max_row 



def segment_skin(image):
  ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

  lower_thresh = (0, 133, 77)
  upper_thresh = (255, 173, 127)

  skin_mask = cv2.inRange(ycrcb, lower_thresh, upper_thresh)

  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
  skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_OPEN, kernel, iterations=3)

  return skin_mask



def face_detection(frame) -> list:
    
    detected_faces = []

    window_sizes = [(64, 64), (96, 96)]

    skin_mask = segment_skin(frame)

    x0, y0, x1, y1 = get_bounding_box(skin_mask)

    for window_size in window_sizes:

        y_stride = window_size[0]
        x_stride = window_size[1]

        for y in range(y0, y1, y_stride):
            for x in range(x0, x1, x_stride):
                window = frame[y : y+window_size[0], x : x+window_size[1]]

                [prediction] = face_classifer.predict( [extract_features_face_detection(window)] )

                if prediction == 1:  
                    detected_faces.append([x, y, x + window_size[1], y + window_size[0]])

    detected_faces = filter_boxes_with_skin_mask(detected_faces, skin_mask)
    detected_faces = union_boxes_min_max(detected_faces, x0, y0, x1, y1)

    return detected_faces


def YAWN_DETECTION_FOR_TRAINED(face,Appear_contours):
    x=face[0]
    y=face[1]
    w=face[2]
    h=face[3]
    global sound_played
    global yawn_counter
    # Enlarge the face circle
    scale_factor = 0.6
    radius = int(max(w, h) * scale_factor)
    center = (x + w // 2, y + h // 2)
    if Appear_contours == 1:
        cv2.circle(frame, center, radius, (255, 0, 0), 2)

    # Define the region of interest for mouth detection 
    roi_face_rgb = rgb_frame[y + h//2 + h//4:y + h, x:x + w]

    # Define the color range for detecting the mouth in RGB
    lower_color = np.array([100, 50, 0], dtype=np.uint8)
    upper_color = np.array([205, 130, 130], dtype=np.uint8)

    # Create mask for the mouth region
    mouth_mask = cv2.inRange(roi_face_rgb, lower_color, upper_color)

    # Find contours in the mask
    contours,_ = cv2.findContours(mouth_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)

        # Check if the bounding box width and height are above a certain threshold
        x_mouth, y_mouth, w_mouth, h_mouth = cv2.boundingRect(largest_contour)
        if w_mouth > 20 and h_mouth > 20:
            # Draw a rectangle around the detected mouth
            if Appear_contours == 1:
                cv2.rectangle(frame, (x + x_mouth, y + h//2 + h//4 + y_mouth), 
                          (x + x_mouth + w_mouth, y + h//2 + h//4 + y_mouth + h_mouth), (0, 255, 0), 1)

            # Check for yawn based on mouth aspect ratio
            mouth_aspect_ratio = calculate_mouth_aspect_ratio(largest_contour)
            if h_mouth > 30 and not sound_played and mouth_aspect_ratio < 1.3 :
                yawn_counter += 1
                cv2.putText(frame, f'Yawn Count: {yawn_counter}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0),
                            2, cv2.LINE_AA)

                if yawn_counter > 40:
                    pygame.mixer.Sound.play(buzzer_sound)
                    sound_played = True  # Set the flag to True after playing the sound
            else:
                yawn_counter = 0
                sound_played = False
 
def YAWN_DETECTION_FOR_HAARCASC(face,Appear_contours):
    global sound_played
    global yawn_counter
    for (x, y, w, h) in face:
        # Enlarge the face circle
        scale_factor = 0.6
        radius = int(max(w, h) * scale_factor)
        center = (x + w // 2, y + h // 2)
        if Appear_contours == 1:
            cv2.circle(frame, center, radius, (255, 0, 0), 2)
        
        # Define the region of interest for mouth detection 
        roi_face_rgb = rgb_frame[y + h//2 + h//4:y + h, x:x + w]

        # Define the color range for detecting the mouth in RGB
        lower_color = np.array([100, 50, 0], dtype=np.uint8)
        upper_color = np.array([205, 130, 130], dtype=np.uint8)

        # Create mask for the mouth region
        mouth_mask = cv2.inRange(roi_face_rgb, lower_color, upper_color)

        # Find contours in the mask
        contours,_ = cv2.findContours(mouth_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Find the contour with the largest area
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)

            # Check if the bounding box width and height are above a certain threshold
            x_mouth, y_mouth, w_mouth, h_mouth = cv2.boundingRect(largest_contour)
            if w_mouth > 20 and h_mouth > 20:
                # Draw a rectangle around the detected mouth
                if Appear_contours == 1:
                    cv2.rectangle(frame, (x + x_mouth, y + h//2 + h//4 + y_mouth), 
                              (x + x_mouth + w_mouth, y + h//2 + h//4 + y_mouth + h_mouth), (0, 255, 0), 1)

                # Check for yawn based on mouth aspect ratio
                mouth_aspect_ratio = calculate_mouth_aspect_ratio(largest_contour)
                if h_mouth > 30 and not sound_played and mouth_aspect_ratio < 1.3 :
                    yawn_counter += 1
                    cv2.putText(frame, f'Yawn Count: {yawn_counter}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0),
                                2, cv2.LINE_AA)
                    
                    if yawn_counter > 40:
                        pygame.mixer.Sound.play(buzzer_sound)
                        sound_played = True  # Set the flag to True after playing the sound
                else:
                    yawn_counter = 0
                    sound_played = False  # Reset the flag if no yawn is detected
     
def EYE_DETECTION_HAARCASC(face):
    for (x, y, w, h) in face:
        roi_gray = gray_frame[y:y + h, x:x + w]
        # Extract the left and right eye coordinates
        leftEye = roi_gray[5:11, 0:w]
        rightEye = roi_gray[5:11, w - 6:w]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        # Calculate the average intensity within the eye regions
        average_intensity_left = np.mean(leftEye)
        average_intensity_right = np.mean(rightEye)

        # Define a threshold for the intensity change
        intensity_threshold = 12

        # Check for sudden changes in intensity
        if abs(average_intensity_left - average_intensity_right) > intensity_threshold:
            cv2.putText(frame, "Awake", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "sleepy", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Draw contours for the eyes inside the detected face region
        leftEyeHull = cv2.convexHull(np.array([(30, 120), (30, 60), (w-30, 120), (w-30, 60)]))
        rightEyeHull = cv2.convexHull(np.array([(30, 120), (30, 60), (w-30, 120), (w-30, 60)]))

        cv2.drawContours(frame[y:y + h, x:x + w], [leftEyeHull], -1, (0, 255, 0), 2)
        cv2.drawContours(frame[y:y + h, x:x + w], [rightEyeHull], -1, (0, 255, 0), 2)
        
        
def EYE_DETECTION_TRAINED_FACE(face):
    x=face[0]
    y=face[1]
    w=face[2]
    h=face[3]
    roi_gray = gray_frame[y:y + h, x:x + w]
    # Extract the left and right eye coordinates
    leftEye = roi_gray[5:11, 0:w]
    rightEye = roi_gray[5:11, w - 6:w]
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)

    ear = (leftEAR + rightEAR) / 2.0

    # Calculate the average intensity within the eye regions
    average_intensity_left = np.mean(leftEye)
    average_intensity_right = np.mean(rightEye)

    # Define a threshold for the intensity change
    intensity_threshold = 12

    # Check for sudden changes in intensity
    if abs(average_intensity_left - average_intensity_right) > intensity_threshold:
        cv2.putText(frame, "Awake", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    else:
        cv2.putText(frame, "sleepy", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Draw contours for the eyes inside the detected face region
    leftEyeHull = cv2.convexHull(np.array([(30, 120), (30, 60), (w-30, 120), (w-30, 60)]))
    rightEyeHull = cv2.convexHull(np.array([(30, 120), (30, 60), (w-30, 120), (w-30, 60)]))

    cv2.drawContours(frame[y:y + h, x:x + w], [leftEyeHull], -1, (0, 255, 0), 2)
    cv2.drawContours(frame[y:y + h, x:x + w], [rightEyeHull], -1, (0, 255, 0), 2)

# 4. Main Application

In [7]:
camera = cv2.VideoCapture(0)
yawn_counter=0
sound_played = False

while camera.isOpened():

    ret, frame = camera.read()
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if HaarCascade == 1:
        face = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
        YAWN_DETECTION_FOR_HAARCASC(face,0)
        EYE_DETECTION_HAARCASC(face)
    else:
        face = face_detection(frame) # face -> (x0, y0, x1, y1)
        YAWN_DETECTION_FOR_TRAINED(face,0)
        EYE_DETECTION_TRAINED_FACE(face)


    #cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), 2)

    cv2.imshow("Stream", frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        print(frame.shape)
        break

camera.release()
cv2.destroyAllWindows()

C:\Users\omar saeed\AppData\Local\Temp\ipykernel_3144\703565035.py:29: RuntimeWarning: divide by zero encountered in double_scalars
  ear = (A + B) / (2.0 * C)


(480, 640, 3)
